# Titanic passengers survival Kaggle competition

In this notebook we will conduct analysis and predictive modelling of Titanic passengers survival dataset for Kaggle [competition](https://www.kaggle.com/c/titanic).

Titanic passengers survival data set is one of the most canonical machine learning datasets. Let's see how this data looks like and try our best to predict survival chances of passenger

The main steps of the analysis process we will conduct are:
1. Frame the problem and look at the big picture
2. Get the data
3. Prepare the data for machine learning algorithms
4. Select a model and train it
5. Fine-tune the model
6. Submit solution